In [ ]:
import numpy as np
import keras
import matplotlib.pyplot as plt
import csv
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import *
from keras.layers import *
from tqdm import tqdm
from multiprocessing import Pool


def image_preprocessing(img):
    """preproccesing training data to keep only S channel in HSV color space, and resize to 16X32"""

    resized = cv2.resize((cv2.cvtColor(img, cv2.COLOR_RGB2HSV))[:,:,:],(img_cols,img_rows))
    return resized

def extract(i):
    X_ = []
    y_ = []

    img_path = logs[i][0]
    img_path = data_folder+'IMG'+(img_path.split('IMG')[1]).strip()
    img = plt.imread(img_path)
    X_.append(image_preprocessing(img))
    y_.append(float(logs[i][3]))

    img_path = logs[i][1]
    img_path = data_folder+'IMG'+(img_path.split('IMG')[1]).strip()
    img = plt.imread(img_path)
    X_.append(image_preprocessing(img))
    y_.append(float(logs[i][3]) + delta_)

    img_path = logs[i][2]
    img_path = data_folder+'IMG'+(img_path.split('IMG')[1]).strip()
    img = plt.imread(img_path)
    X_.append(image_preprocessing(img))
    y_.append(float(logs[i][3]) - delta_)

    return X_, y_

def load_data(X,y,data_folder,delta=0.08):
    """function to load training data"""

    global logs
    global delta_

    delta_ = delta

    log_path = data_folder + 'driving_log.csv'
    logs = []

    # load logs
    with open(log_path,'rt') as f:
        reader = csv.reader(f)
        for line in reader:
            logs.append(line)
        log_labels = logs.pop(0)

    pool = Pool(processes=6)

    for X_, y_ in tqdm(pool.imap_unordered(extract, range(len(logs))), total=len(logs)):
        X.extend(X_)
        y.extend(y_)
    pool.close()

In [ ]:
img_rows = 120
img_cols = 240

#data path
data_folder = '../../data/udacity_official/'

#batch size and epoch
batch_size=128
nb_epoch=7

data={}
data['features'] = []
data['labels'] = []

load_data(data['features'], data['labels'], data_folder, 0.3)

X_train = np.array(data['features']).astype('float32')
y_train = np.array(data['labels']).astype('float32')


# horizonal reflection to agument the data
X_train = np.append(X_train, X_train[:,:,::-1,:], axis=0)
y_train = np.append(y_train, -y_train, axis=0)

# split train and validation
X_train, y_train = shuffle(X_train, y_train)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0, test_size=0.1)

# reshape to have correct dimension
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 3)


In [ ]:
from keras.models import Sequential, Model
from keras.layers.core import Lambda, Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Cropping2D, Convolution2D, MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers.noise import GaussianNoise
from keras.optimizers import Adam

print("\nBuilding and compiling the model ...")

model = Sequential()
# Preprocess incoming data, centered around zero with small standard deviation 
model.add(Lambda(lambda x: (x / 127.5) - 1.0, input_shape=(img_rows, img_cols, 3)))
# Block - conv
model.add(Convolution2D(2, 11, 11, border_mode='valid', subsample=[2, 2], init='glorot_uniform', activation='relu', name='Conv1'))
# Block - conv
model.add(Convolution2D(1, 5, 5, border_mode='valid', subsample=[2, 2], init='glorot_uniform', activation='relu', name='Conv2'))
# Block - conv
model.add(Convolution2D(1, 3, 3, border_mode='valid', subsample=[2, 2], init='glorot_uniform', activation='relu', name='Conv3'))
# Block - conv
#model.add(Convolution2D(1, 3, 3, border_mode='valid', init='glorot_uniform', activation='relu', name='Conv4'))
# Block - flatten
#model.add(MaxPooling2D((4,4),(4,4),'valid', name='pool1'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(1, name='output')) 
model.summary()

adam = Adam(lr=0.001)
model.compile(loss='mean_squared_error', optimizer=adam)

In [ ]:
AUGMENTATION_FACTOR = 1
nb_epoch=20

import keras
from keras.callbacks import Callback
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
import math


tensorboard = keras.callbacks.TensorBoard(log_dir='../../data/logs/steer_only/', histogram_freq=1, 
                                          write_graph=True, write_images=True)
checkpoint = keras.callbacks.ModelCheckpoint("../../data/weights/steer_only/model.h5", monitor='val_loss', verbose=0, 
                                             save_best_only=True, save_weights_only=True, mode='auto', period=1)

 
history = model.fit(X_train, y_train, 
                  batch_size=batch_size,
                  nb_epoch=nb_epoch, verbose=1,
                  validation_data=(X_val, y_val))

print("\nTraining the model ended.")

print("\nSaving Model...")

model_json = model.to_json()
with open("../../data/weights/steer_only/model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("../../data/weights/steer_only/model.h5")
print("Model Saved.")

In [ ]:
print("\nSaving Model...")

model_json = model.to_json()
with open("../../data/weights/steer_only/model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("../../data/weights/steer_only/model.h5")
print("Model Saved.")